In [1]:
# Import required Python packages
import numpy as np # version 1.18.5
import networkx as nx # version 2.4
#import community # version 0.13 (python-louvain)
import gudhi # version 3.3.0
import itertools
import math
import random
from statistics import quantiles
from statistics import stdev
from statistics import mean
import scipy.stats as ss

import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.cm as cm
from matplotlib.colors import Normalize

import scipy.io # version 1.4.1
from sklearn import preprocessing # version 0.23.1

import seaborn as sns
import pandas as pd
import glob
from pathlib import Path
import time

get_ipython().run_line_magic('load_ext', 'watermark')

import Simplicial_Complexes as sc
import Spatial_distributions as sd
import os

# Create Graph class
class Graph(object):

    # Initialize the matrix
    def __init__(self, size):
        self.adjMatrix = []
        for i in range(size):
            self.adjMatrix.append([0 for i in range(size)])
        self.size = size

    # Add edges
    def add_edge(self, v1, v2):
        if v1 == v2:
            print("Same vertex %d and %d" % (v1, v2))
        self.adjMatrix[v1][v2] = 1
        self.adjMatrix[v2][v1] = 1

    # Remove edges
    def remove_edge(self, v1, v2):
        if self.adjMatrix[v1][v2] == 0:
            print("No edge between %d and %d" % (v1, v2))
            return
        self.adjMatrix[v1][v2] = 0
        self.adjMatrix[v2][v1] = 0

    def __len__(self):
        return self.size

    # Print the matrix
    def print_matrix(self):
        print(np.array(self.adjMatrix))


def valid_vertices(A,n_vertices):
    vertices_v = list()
    for i in range(n_vertices):
        boolean = False
        for edges in A:
            for edge in edges:
                if i in edge:
                    boolean = True
        if boolean:
            vertices_v.append(i)
    return vertices_v


In [ ]:
# Visualization options
options = {"node_color": "grey", "edgecolors": "black", "alpha": 0.9}
options2 = {"node_size": 10, "node_color": "black", "edgecolors": "black", "alpha": 0.9}

# Folder to save the results
folder_name = "Example"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
# Parameters
field_extension = 5 #Maximum entension of the synaptic field to create an edge
min_d = 2 #Minimum dimension of a simplex to make a neuron fire
max_D = 7 #Maximum dimension of a simplex in this simulation
sigma = 3
n_vertices = 20
xmin = 0; xmax = 10
ymin = 0; ymax = 10

# To clarify our notation, we will use data when it is a list and points when it is a np.array
points = sd.discgaussianN_2d(xmax,ymax,n_vertices,sigma)
data = list(map(tuple, points))

# Plot the current spatial distribution of cells
plt.figure()
sd.plot_points(points,0,max(points[:,0]),0,max(points[:,1]),folder_name)
plt.close()

# Calculate the Cech complex of the entire network
position_dict = {i:data[i] for i in range(n_vertices)} # To specify the position of our nodes in networkx package
cech_complex = sc.SimplicesCechComplex(position_dict, field_extension, max_D)
st = gudhi.SimplexTree()
for i in range(n_vertices):
    st.insert([i],1.00)
for simplex in cech_complex:
    st.insert(simplex, float(len(simplex)))

# Visualize the results
g = Graph(n_vertices)
result_str = 'Simplicial complex is of dimension ' + repr(st.dimension()) + ' - ' +     repr(st.num_simplices()) + ' simplices - ' +     repr(n_vertices) + ' vertices.'
filtration = st.get_filtration()
with open(os.path.join(folder_name,"Resulting_SimplicialComplex.txt"),'w+') as file:
    file.write(result_str)
    file.write('\nAbstract simplicial complex: (Simplex -> Filtration value)\n')
    for filtered_value in filtration:
        file.write("%s -> %.2f\n" % tuple(filtered_value))
        simplex = filtered_value[0]
        if(len(simplex)==2):
            g.add_edge(simplex[0],simplex[1])

A = np.array(g.adjMatrix)
G = nx.from_numpy_matrix(A)  

valid_edges = list()
nx.draw_networkx_nodes(G, position_dict, **options)
nx.draw_networkx_edges(G, position_dict, edge_color='k', alpha=0.7) # Background network with all edges

filtration = st.get_filtration()

for filtered_value in filtration:
    simplex = filtered_value[0]
    dim = len(simplex)
    nsimplex_list[dim-1] = nsimplex_list[dim-1] + 1
    if dim >= min_d:
        edges = list(itertools.combinations(simplex, 2)) #Return 2 length subsequences of elements from the input simplex.
        valid_edges.append(edges)
        nx.draw_networkx_edges(
            G,
            position_dict,
            edgelist=edges,
            width=1.5,
            edge_color = 'r'
        )

labels = {i:i for i in range(n_vertices)}
nx.draw_networkx_labels(G, position_dict, labels, font_color="whitesmoke")

plt.title('Geometrical simplicial complex',fontsize=16)
plt.grid()
plt.draw()
plt.savefig(os.path.join(folder_name,"Geometrical_SC.png"))
plt.savefig(os.path.join(folder_name,"Geometrical_SC.svg"))
plt.close()

plt.figure()
diag = st.persistence(min_persistence=-1)
plt.close()

nholes_list = np.zeros(10)
nholes_list = st.betti_numbers()

with open(os.path.join(folder_name,"Nholes.txt"),'w+') as file:
    file.write('\nDimension of hole -> Number of holes\n')
    for count,value in enumerate(nholes_list):
        file.write("%s -> %s\n" % tuple((count,value)))

if len(valid_edges)>0:    
    # FUNCTIONAL NETWORK
    #Show only the functional interactions

    vertices_v = valid_vertices(valid_edges,n_vertices)
    points_v = points[vertices_v]
    data_v = list(map(tuple, points_v))
    position_dict_v = {i:data_v[i] for i in range(len(points_v))} 
    cech_complex_v = sc.SimplicesCechComplex(position_dict_v, field_extension, max_D)

    nsimplex_list_v = np.zeros(max_D)

    st_v = gudhi.SimplexTree()
    for val in range(len(points_v)):
        st_v.insert([val],1.00)

    for simplex in cech_complex_v:
        dim = len(simplex)
        st_v.insert(simplex, float(dim))
        nsimplex_list_v[dim-1] = nsimplex_list_v[dim-1] + 1

    with open(os.path.join(folder_name,"Nsimplex_functional.txt"),'w+') as file:
        file.write('\nDimension of simplex -> Number of simplices\n')
        for count,value in enumerate(nsimplex_list_v):
            file.write("%s -> %s\n" % tuple((count,value)))

    n_vertices_v = len(vertices_v)

    result_str = 'Functional simplicial complex is of dimension ' + repr(st_v.dimension()) + ' - ' +     repr(st_v.num_simplices()) + ' simplices - ' +     repr(n_vertices_v) + ' vertices.'
    filtration = st_v.get_filtration()
    with open(os.path.join(folder_name,"Functional_SimplicialComplex.txt"),'w+') as file:
        file.write(result_str)
        file.write('\nAbstract functional simplicial complex: (Simplex -> Filtration value)\n')
        for filtered_value in filtration:
            file.write("%s -> %.2f\n" % tuple(filtered_value))

#Visualize matrix as graph, highlighting the relevant simplices 
    position_dict = {i:data_v[i] for i in range(n_vertices_v)} # To specify the position of our nodes in networkx package
    plt.figure()
    diag = st_v.persistence(min_persistence=-1)
    plt.close()

    nholes_list_v = st_v.betti_numbers()

    with open(os.path.join(folder_name,"Nholes_functional.txt"),'w+') as file:
        file.write('\nDimension of hole -> Number of holes\n')
        for count,value in enumerate(nholes_list_v):
            file.write("%s -> %s\n" % tuple((count,value)))
    del st_v
del st

N20


usetex mode requires TeX.
C:\Users\alext\anaconda3\lib\site-packages\gudhi\persistence_graphical_tools.py:160: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(1, 1)
<ipython-input-2-9a595101094b>:55: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
<ipython-input-2-9a595101094b>:89: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the 

1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
N25
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
N30
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
N35
